# Cython basics

## Purpose

The following are notes and excerpts from this author's learning of Cython. We begin by following http://docs.cython.org/ and proceed elsewhere if needed (in which case, related sources will be disclosed at that time).

Note that in Jupyter notebooks, we can load a Cython magic function by

In [1]:
%load_ext Cython

Then the cell magic %%cython will allow cython builds to be completed by Jupyter. 

Building Cython modules can happen be done otherwise by two principal means. First we note that a cython module should have a suffice <code>pyx</code>. In two steps we first generate a <code>.c</code> file, then we use this C file to generate a <code>.so</code> file.

<ol>
<li>Creating a build script <code>setup.py</code> which uses <code>distutils</code> to automate the procedure.

<li> Manually, by the following
$${}$$
    > cython yourfile.pyx
$${}$$
    > gcc -shared -pthread -fPIC -fwrapv -O2 -Wall -fno-strict-aliasing -I/usr/include/python2.7 -o yourfile.so youfile.c
    
</ol>

As concerns item 1, a <code>setup.py</code> has the following prototype

In [ ]:
# example setup.py

from distutils.core import setup
from distutils.extension import Extension
from Cython.Build import cythonize

ext_modules=[
    Extension("yourfile",
              sources=["yourfile.pyx"],
              libraries=["m"] # Unix-like specific
    )
]

setup(
  name = "yourfile",
  ext_modules = cythonize(ext_modules)
)


The previous <code>setup.py</code> may have extraneous details, e.g. the inclusion of <code>"libraries = ["m"]"</code> is used. In general, multiple libraries can be indicated in the build script. In this example, we suppose we have used in our cython module <code>yourfile.pyx</code> a function from the C-library <code>math.h</code>. In other words, the libraries keyword amounts to compiling with the math library with the linking <code>-m</code>, i.e.

    > gcc yourfile.c -m
    
compiles by linking the math library (note that the order of the tag is sensitive. It has been a historic inconsistency where the user must place the <code>-m</code> flag on the compiler call. On different versions of <code>gcc</code>, it very well can be <code>gcc -m yourfile.c</code> that must be used. The compile will not fail, but when trying to execute the file a reference to a math library function an error notice may be echoed (e.g. with flags that provide more information <code>gcc -Wall -Wextra -m yourfile.c</code>) such as <code>"undefined reference to sin function"</code>.

# Function basics

### Function objects and scope

We define Python objects through <code>def</code>, we define Cython objects with <code>cdef</code>. <code>cdef</code> objects cannot be used interactively or exported outside of their Cython module (<code>.pyx</code> file). Python objects can be used interactively and exported outside their module as needed. If we require using a Cython function outside of its module we have two options:

<ul>
<li><code>cpdef</code>: hybrid function that permits python calling from non-cython modules (slow), and also C calling when called within the same cython module (fast). This dual typing comes at the cost of a claimed "tiny" overhead increase. Note that python function calls have substantial overhead cost compared to C style. It is possible that python instantiations (for example) calling a particular function may call the python version instead of the C version, hence in such cases the overhead savings is, of course, not possible.
<li>we can wrap a cython function in a python <code>def</code> definition. Alternatively, we seem to be able to use so-called <code>cdef classes</code>. We will read about this later.
</ul>

### variable typing

Typing variables and functions can produce signficant speed increases. Functions of either kind can take C data types as arguments. This comes at the cost of conversion between C objects and Python objects. Consider the python function

In [ ]:
def spam(int i, char *s):
    ...

cdef int eggs(unsign

which takes in C data types. The objects are actually handled as:

In [ ]:
def spam(python_i, python_s):
    cdef int i = python_i
    cdef char* s = python_s
    ...

i.e. these are passed as python objects, then converted to C types. <b>Only numeric types can be handled consistently between C and Python types, e.g. strings are at this time not handled</b>

C functions can have parameters of any type. Further, objects that are undeclared and passed as parameters are interpreted as Python objects (<i>not</i> int as would be the case in C). Thus, the default is always Python, not C.

<code>cdef</code> functions will assign a return value of <code>False</code> in Cython should the function body not contain an explicit return value. In C, of course, such a situation would leave the return value as undefined (e.g. <code>void</code> return).

### exceptions

"If you don’t do anything special, a function declared with cdef that does not return a Python object has no way of reporting Python exceptions to its caller. If an exception is detected in such a function, a warning message is printed and the exception is ignored."

We designate a certain return as an exception that can communicate to the caller, e.g. the int -1 by the special prototype:

In [ ]:
cdef int spam() except -1:
    ...

If all such returns are legal, then we can still flag a designated return, e.g. the int -1, by the following

In [ ]:
cdef int space() except? -1:
    ...

The program runs without interruption, but Cython calls <code>PyErr_Occurred()</code> to amke sure the error really is an error. Note that the return type of function can have the int return flag -1 even if it is <code>int, enum, float</code>, or <code>pointer</code> type. A third option which works for all the aforemntioned return types as well as the only one that works for <code>void</code> returns is to call <code>PyErr_Occurred()</code> after every such function call per:

In [ ]:
cdef int spam() except *:
    ...

i.e. it is the only exception prototype that works with <code>void</code>, that is

In [ ]:
cdef void spam() except *:
    ...

"You don’t need to (and shouldn’t) declare exception values for functions which return Python objects. Remember that a function with no declared return type implicitly returns a Python object. (Exceptions on such functions are implicitly propagated by returning NULL.)"

## Differences between Cython and C

<ul>
<li>There is no <code>-></code> operator in Cython, instead of <code>p->x</code> use <code>p.x</code>
<li>There is no derefencing operator <code>*</code> for arrays, instead of <code>*p</code> use <code>p[0]</code>. Accordingly, pointer arithemtic, e.g. <code>*(p + 1)</code> will not produce <code>p[1]</code> as it would in C.
<li><code>NULL</code> is not the same as <code>0</code>. <code>NULL</code> is a reserved word and is used for null C pointers. That is, you cannot use C idioms like <code>if (!p)</code> to check for null pointers since <code>!p<code> does not evaluate to <code>True</code> as it would in C if <code>p</code> was NULL, i.e. having the same value as <code>0</code>.
<li>Casting is done with <code><></code>, not <code>()</code>.

Example

    cdef char * p, float * q
    p = <char *> q

## Note on built-in functions

"Cython compiles calls to most built-in functions into direct calls to the corresponding Python/C API routines, making them particularly fast."

These calls must be done to the actual names of built-ins, i.e. you cannot assign a python object to a function and have the call to that python name inherit the same efficiency, as it now interfaces with python, not C.

## Integer for-loops

for the Pythonic loop

In [ ]:
for i in range(n):
    ...

Cython will optimize this into a pure C loop if <code>i</code> is declared as <code>cdef int</code> type. The following, in suit with the legacy version Pyrex is permissible (though <b>deprecated</b>):

In [ ]:
for i from 0 <= i < n:
    ...

or

In [ ]:
for i from 0 <= i < n by s:
    ...

Foir some step size integer <code>s</code>.

## Include directives

In [ ]:
include "spamstuff.pxi"

includes statements listed in the indicated file. For example, declarations, function declarations.

Instead of the C style <code>#IFNDEF</code> conditional statements, python style syntax is used at the zero indent level:

In [ ]:
IF UNAME_SYSNAME == "Windows":
    include "icky_definitions.pxi"
ELIF UNAME_SYSNAME == "Darwin":
    include "nice_definitions.pxi"
ELIF UNAME_SYSNAME == "Linux":
    include "penguin_definitions.pxi"
ELSE:
    include "other_definitions.pxi"